In [ ]:
import numpy as np
import pandas as pd

import os
import math
import random
import cv2

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid

from PIL import Image

import seaborn as sns

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

%matplotlib inline

# Quick Look at the Data

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['tag_set'] = df['tags'].map(lambda s: set(s.split(' ')))

tags = set()
for t in df['tags']:
    s = set(t.split(' '))
    tags = tags | s

tag_list = list(tags)
tag_list.sort()
tag_columns = ['tag_' + t for t in tag_list]
for t in tag_list:
    df['tag_' + t] = df['tag_set'].map(lambda x: 1 if t in x else 0)

In [ ]:
df.info()
df.describe()

In [ ]:
df.head()

In [ ]:
df[tag_columns].sum()

In [ ]:
df[tag_columns].sum().sort_values().plot.bar()

In [ ]:
tags_count = df.groupby('tags').count().sort_values(by='image_name', ascending=False)['image_name']
print('There are {} unique tag combinations'.format(len(tags_count)))
print()
print(tags_count)

In [ ]:
from textwrap import wrap

def display(images, cols=None, maxcols=10, width=16, titles=None):
    if cols is None:
        cols = len(images)
    n_cols = cols if cols < maxcols else maxcols
    plt.rc('axes', grid=False)
    fig1 = plt.figure(1, (width, width * math.ceil(len(images)/n_cols)))
    grid1 = ImageGrid(
                fig1,
                111,
                nrows_ncols=(math.ceil(len(images)/n_cols), n_cols),
                axes_pad=(0.1, 0.6)
            )

    for index, img in enumerate(images):
        grid1[index].grid = False
        if titles is not None:
            grid1[index].set_title('\n'.join(wrap(titles[index], width=25)))
        if len(img.shape) == 2:
            grid1[index].imshow(img, cmap='gray')
        else:
            grid1[index].imshow(img)

In [ ]:
from skimage import io

def load_image(filename, resize=True, folder='train-jpg'):
    img = io.imread('{}/{}.jpg'.format(folder, filename))
    #print(img.shape)
    #r, g, b, nir = img[:, :, 0], img[:, :, 1], img[:, :, 2], img[:, :, 3]
    #print(nir.min(), nir.max())
    if resize:
        img = cv2.resize(img, (224, 224))
    return np.array(img)

def mean_normalize(img):
    work = img.copy().astype(np.float32)
    for c in range(work.shape[-1]):
        w = work[...,c]
        work[...,c] = (w - w.mean()) / (w.max() - w.min())
    return work

def normalize(img):
    return img / 127.5 - 1

def enhance(img):
    work = img.copy()
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    for c in range(work.shape[-1]):
        work[..., c] = clahe.apply(work[..., c])
    return work

def unnormalize(img):
    work = img.copy()
    for c in range(work.shape[-1]):
        w = work[...,c]
        work[...,c] = 255 * (w - w.min()) / (w.max() - w.min())
    return work.astype(np.uint8)

In [ ]:
#nir = sample_images[0][:,:,3]
#nir.max()

In [ ]:
samples = df.sample(36)
sample_images = [load_image(fn) for fn in samples['image_name']]
INPUT_SHAPE = sample_images[0].shape
print(INPUT_SHAPE)
display(
    sample_images,
    cols=6,
    titles=[t for t in samples['tags']]
)

In [ ]:
def my_preprocess(img):
    img = mean_normalize(img)
    return img

preprocess = my_preprocess

si = np.array(sample_images[:2])
display(unnormalize(preprocess(si)))

In [ ]:
display(
    [unnormalize(preprocess(img)) for img in sample_images],
    cols=6,
    titles=[t for t in samples['tags']]
)

# Learn

In [ ]:
# df_train <- select up to 300 samples per tag
samples_per_tag = 300
df_train = []
for t in tag_columns:
    if t in ('tag_primary', 'clear'):
        continue
    tag_condition = df[t] == 1
    take = min(samples_per_tag, tag_condition.sum())
    df_train.append(df[tag_condition].sample(take).values)

df_train = np.concatenate(df_train)
df_train = pd.DataFrame(df_train)
df_train.columns = df.columns

df_train[tag_columns].sum()
df_train = df

In [ ]:
X = df_train['image_name'].values
y = df_train[tag_columns].values

n_features = 1
n_classes = y.shape[1]

X, y = shuffle(X, y)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

print('We\'ve got {} feature rows and {} labels'.format(len(X_train), len(y_train)))
print('Each row has {} features'.format(n_features))
print('and we have {} classes'.format(n_classes))
assert(len(y_train) == len(X_train))
print('We use {} rows for training and {} rows for validation'.format(len(X_train), len(X_valid)))
print('Each image has the shape:', INPUT_SHAPE)
print('So far, so good')

In [ ]:
print('Memory usage (train) kB', X_train.nbytes//(1024))
print('Memory usage (valid) kB', X_valid.nbytes//(1024))

In [ ]:
def fliplr(img, y):
    return np.fliplr(img.copy()), y

def flipud(img, y):
    return np.flipud(img.copy()), y

def adjust_brightness(img, y, amount):
    result = img.astype(np.int16)
    result += amount
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result, y

def _augment(X, y):
    new_X, new_y = [], []
    def add(nx, ny):
        new_X.append(nx)
        new_y.append(ny)

    amount = int(random.uniform(5, 15))
    add(X, y)
    add(*fliplr(X, y))
    add(*flipud(X, y))
    add(*flipud(*fliplr(X, y)))
    add(*adjust_brightness(*fliplr(X, y), amount))
    add(*adjust_brightness(*flipud(X, y), -amount))
    add(*adjust_brightness(*(X, y), amount))
    add(*adjust_brightness(*(X, y), -amount))

    return new_X, new_y

def generator(X, y, augment=False, batch_size=32):
    X_copy, y_copy = X, y
    while True:
        for i in range(0, len(X_copy), batch_size):
            X_result, y_result = [], []
            for x, y in zip(X_copy[i:i+batch_size], y_copy[i:i+batch_size]):
                rx, ry = load_image(x), y

                if augment:
                    rx, ry = _augment(rx, ry)
                else:
                    rx, ry = [rx], [ry]

                rx = np.array([preprocess(x) for x in rx])
                ry = np.array(ry)
                X_result.append(rx)
                y_result.append(ry)
            X_result, y_result = np.concatenate(X_result), np.concatenate(y_result)
            yield shuffle(X_result, y_result)
        X_copy, y_copy = shuffle(X_copy, y_copy)

In [ ]:
from keras import backend as K

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

# ---------------------------------- #

def my_model():
    model = Sequential()

    model.add(Conv2D(48, (8, 8), strides=(2, 2), input_shape=INPUT_SHAPE, activation='elu'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (8, 8), strides=(2, 2), activation='elu'))
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3, 3), activation='elu'))
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3, 3), activation='elu'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(0.3))

    model.add(Dense(256, activation='elu'))
    model.add(BatchNormalization())

    model.add(Dense(64, activation='elu'))
    model.add(BatchNormalization())

    model.add(Dense(n_classes, activation='sigmoid'))
    return model

def resnet_model():
    base = ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    
    for layer in base.layers:
        layer.trainable = False
    
    x = Flatten()(base.output)
    x = Dropout(0.3)(x)
    
    x = Dense(256, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='elu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    preds = Dense(n_classes, activation='sigmoid')(x)
    
    head = Model(inputs=base.input, outputs=preds)

    return head
    
model = resnet_model()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[fbeta, 'accuracy']
)

model.summary()

In [ ]:
filepath="resnet-weights-improvement-{epoch:02d}-{val_fbeta:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_fbeta', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
EPOCHS = 4
BATCH = 16
PER_EPOCH = 128

X_train, y_train = shuffle(X_train, y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)

history = model.fit_generator(
    generator(X_train, y_train, augment=False, batch_size=BATCH),
    steps_per_epoch=PER_EPOCH,
    epochs=EPOCHS,
    validation_data=generator(X_valid, y_valid, batch_size=BATCH),
    validation_steps=len(y_valid)//(2*4*BATCH),
    callbacks=callbacks_list
)

In [ ]:
#model.save('model.h5')
model.load_weights('resnet-weights-improvement-02-0.869.hdf5')

In [ ]:
%%time
from tqdm import tqdm

X_test = os.listdir('test-jpg')
X_test = [fn.replace('.jpg', '') for fn in X_test]

result = []
TEST_BATCH = 24
for i in tqdm(range(0, len(X_test), TEST_BATCH)):
    X_batch = X_test[i:i+TEST_BATCH]
    X_batch = np.array([preprocess(load_image(fn, folder='test-jpg')) for fn in X_batch])
    p = model.predict(X_batch)
    result.append(p)
#print(result)

In [ ]:
r = np.concatenate(result)
r = r > 0.5
r

In [ ]:
table = []
for row in r:
    t = []
    for b, v in zip(row, tag_columns):
        if b:
            t.append(v.replace('tag_', ''))
    table.append(' '.join(t))
#list(zip(X_test, table))

In [ ]:
df_pred = pd.DataFrame.from_dict({'image_name': X_test, 'tags': table})
df_pred.to_csv('submission7.csv', index=False)